In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [3]:
train.head()

NameError: name 'train' is not defined

In [4]:
train.isnull().sum()

NameError: name 'train' is not defined

## Geographical plotting/data - Questions to Nelli

In [5]:
plt.figure(figsize = (15,8))
sns.scatterplot(train['X'], train['Y'], alpha=0.2)
plt.xlim(-122.55, -122.35)
plt.ylim(37.7,37.82)

NameError: name 'train' is not defined

<Figure size 1080x576 with 0 Axes>

In [6]:
# Uncomment and run when running the code the first time (or do the installation directly in your terminal)
# !pip install folium

In [7]:
train['Category'].unique()

NameError: name 'train' is not defined

In [8]:
prost = train[train['Category'] == 'PROSTITUTION']

NameError: name 'train' is not defined

In [9]:
prost.shape

NameError: name 'prost' is not defined

In [10]:
bribe = train[train['Category'] == 'BRIBERY']

NameError: name 'train' is not defined

In [11]:
import folium

In [12]:
fake_df = train[:20]

NameError: name 'train' is not defined

In [13]:
fake_df.head()

NameError: name 'fake_df' is not defined

In [14]:
%%time
map_pickup = folium.Map(location=[37.8,-122.5])
fake_df.apply(lambda row:folium.CircleMarker(location=[row['Y'], row['X']]).add_to(map_pickup), axis=1) #try these with bribe and prost for fun

map_pickup

NameError: name 'fake_df' is not defined